In [25]:
from herbie import Herbie
from datetime import datetime, timedelta
import math
    
file_name = '/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet'
data_index = 1 #index starts at 0
start_time_str = '2023-11-06 00:00:00'
ss = ":TMP:2 m|:RH:2 m|:WIND:10 m|:.GRD:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface"
ss1 = ":TMP:2 m|:RH:2 m" # 2m readings
ss2 = ":WIND:10 m|:.GRD:10 m" # 10m readings
ss3 = ":DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface" # surf readings
ll = [(-119.02,37.64),(-119.25,37.91),(-118.73,37.45)] # lat/long order: CUES, Tioga, Rock_Creek

for i in range(1): #to get 48hr model data range=(49)
    dt = datetime.fromisoformat(start_time_str)
    dt_query_str = dt.strftime('%Y-%m-%d %H:%M')
    output_timestamp = dt + timedelta(hours=i)
    output_timestamp_str = output_timestamp.strftime('%Y-%m-%dT%H:%M')
    hours_offset = i
    H = Herbie(start_time_str,model="hrrr",fxx=hours_offset,save_dir="~/ESAC_herbiedata")
    #print(H.inventory(searchString=":TMP:2 m|:RH:2 m|:WIND:10 m|:.GRD:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface"))
    # not sure about winds
    ds1 = H.xarray(ss1, remove_grib=False)
    y1=ds1.herbie.nearest_points(points=ll)
    print('querying data for: ', dt_query_str, hours_offset, output_timestamp_str)
    y1_d = y1.to_dict()
    ds2 = H.xarray(ss2, remove_grib=False)
    y2=ds2.herbie.nearest_points(points=ll)
    print('querying data for: ', dt_query_str, hours_offset, output_timestamp_str)
    y2_d = y2.to_dict()
    append_data(y1_d, y2_d, output_timestamp_str)
# pri#nt(ds)
#print(y)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Nov-06 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "t2m".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "r2".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()


querying data for:  2023-11-06 00:00 0 2023-11-06T00:00


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "u10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "v10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "si10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()


querying data for:  2023-11-06 00:00 0 2023-11-06T00:00
<class 'list'>
33
33


In [23]:
def append_data(y1_d, y2_d, output_timestamp_str):
    t2m = y1_d['data_vars']['t2m']['data'][data_index]
    r2 = y1_d['data_vars']['r2']['data'][data_index]
    maxwind10 = y2_d['data_vars']['si10']['data'][data_index]
    uwind10 = y2_d['data_vars']['u10']['data'][data_index]
    vwind10 = y2_d['data_vars']['v10']['data'][data_index]
    wind10 = math.sqrt(uwind10**2 + vwind10**2)
    t2m_formatted = round(t2m, 1) #2nd parameter is number of decimals
    r2_formatted = round(r2)
    maxwind10_formatted = round(maxwind10, 1)
    wind10_formatted = round(wind10, 1)
    new_line_str = f"{output_timestamp_str} {t2m_formatted} {r2_formatted} {maxwind10_formatted} {wind10_formatted}\n"
    with open(file_name, 'r') as f:
        lines = f.readlines()
        print(type(lines))
        print(len(lines))
        new_lines = lines
        found= False
        for idx, line in enumerate(lines):
            if line[:16] == output_timestamp_str:
                new_lines[idx] = new_line_str
                found = True
                break
        if not found:
            new_lines += [new_line_str]
    print(len(new_lines))
    with open(file_name, 'w') as f:
        f.writelines(new_lines)
        
print("done")

done


In [11]:
from herbie import Herbie
H = Herbie(start_time_str,model="hrrr",fxx=hours_offset,save_dir="~/ESAC_herbiedata")
ds2 = H.xarray(ss2, remove_grib=False)
y2=ds2.herbie.nearest_points(points=ll)
print(y2)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2023-Nov-06 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:260: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.parse_cf()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "u10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "v10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "si10".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gribfile_projection".
  ds = ds.metpy.assign_y_x()


<xarray.Dataset>
Dimensions:              (point: 3)
Coordinates:
    time                 datetime64[ns] 2023-11-06
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 10.0
    latitude             (point) float64 37.65 37.91 37.45
    longitude            (point) float64 241.0 240.8 241.3
    valid_time           datetime64[ns] 2023-11-06
    metpy_crs            object Projection: lambert_conformal_conic
    y                    (point) float64 1.257e+05 1.587e+05 9.869e+04
    x                    (point) float64 -1.879e+06 -1.891e+06 -1.858e+06
  * point                (point) int64 0 1 2
    point_latitude       (point) float64 37.64 37.91 37.45
    point_longitude      (point) float64 -119.0 -119.2 -118.7
Data variables:
    u10                  (point) float32 7.55 14.05 8.8
    v10                  (point) float32 5.7 5.138 2.2
    si10                 (point) float32 9.441 14.94 9.066
    gribfile_projection  (point) object None None None
Attrib